# Double box version

In [1]:
import cv2
import glob
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.externals import joblib
from keras.models import model_from_json
from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label
from skimage.feature import local_binary_pattern

Using TensorFlow backend.


In [2]:
# Metaparameters
colorspace= 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient= 12
pix_per_cell= 4
cell_per_block= 1 #2
hog_channel= 'ALL' # Can be 0, 1, 2, or "ALL"


In [3]:
# Customized re-implementation of HOG, based on OpenCV and scikit-image versions
def histoGrad(img, binCount=12, cellSize=4):
    gx,gy= cv2.Sobel(img,cv2.CV_32F,1,0), cv2.Sobel(img,cv2.CV_32F,0,1) # gradients in x and y direction
    magnitude,angle= cv2.cartToPolar(gx,gy)
    bins= np.int32(binCount*angle*(1-1e-7)/(2*np.pi)) #scale back angle to avoid overbinning
    binCells,magCells= [],[]
    cx= cy= cellSize
    for i in range(0,int(img.shape[0]/cy)): # sort into bins
        for j in range(0,int(img.shape[1]/cx)):
            binCells.append(bins[i*cy:i*cy+cy, j*cx:j*cx+cx])
            magCells.append(magnitude[i*cy:i*cy+cy, j*cx:j*cx+cx])
    hist= np.hstack([np.bincount(i.ravel(), j.ravel(), binCount) for i,j in zip(binCells, magCells)])
    return np.sqrt(hist/(hist.sum()+1e-8)) # apply Hellinger kernel (L1-normalize and sqrt)

In [4]:
# Function to return HOG features
def get_hog_features(img, orient, pix_per_cell):
    features= histoGrad( img, orient, pix_per_cell )
    return features.reshape((orient*pix_per_cell*pix_per_cell,))    #192

# Function to return color histogram features  
def get_color_hist(img, nbins=32, bins_range=(0, 32)):
    return np.histogram(img, bins=nbins, range=bins_range)

# Function to return spatial binning of color features
def get_spatial(img, size=(8, 8)):
    return np.ravel(cv2.resize(img, size, interpolation=3) )/255.

# Function to return texture features
def get_texture_features(img, quantization=8, radius=3):
    lbp= np.ravel(local_binary_pattern(img, quantization, radius, method='uniform'))
    return np.mean(lbp.reshape(-1, 4), axis=1) # average pooling

In [5]:
# Load binary classifier model

model= None
with open('model.json', 'r') as jfile:
    model = model_from_json(json.loads(jfile.read()))
model.compile("adam", "binary_crossentropy")
#weights_file = args.model.replace('json', 'h5')
model.load_weights('model.h5')
model.summary()
X_scaler= joblib.load('scalar.dat')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dropout_1 (Dropout)              (None, 624)           0           dropout_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 255)           159375      dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 255)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 127)           32512       dropout_2[0][0]                  
___________________________________________________________________________________________

In [6]:
def getFeaturesFromColorPatch(patch, orient=12, pix_per_cell=4, cell_per_block=1):
    global firstTime
    hog_features = []
    feature_patch= cv2.cvtColor(patch, cv2.COLOR_RGB2YCrCb)
    for channel in range(feature_patch.shape[2]):
        #hog_features.append(get_hog_features(feature_patch[:,:,channel], 
        #                    orient, pix_per_cell, cell_per_block, 
        #                    vis=False, feature_vec=True))
        
        # Compute gradient AND small color patch (note: color scaling)
        hog_features.append(np.concatenate((get_hog_features(feature_patch[:,:,channel], orient, pix_per_cell), 
            np.ravel(cv2.resize(feature_patch[:,:,channel], (4,4), interpolation=3) )/255. ))) #CV_INTER_AREA
        #hog_features.append(np.concatenate((get_hog_features(feature_patch[:,:,channel], 
        #                    orient, pix_per_cell, cell_per_block, 
        #                    vis=False, feature_vec=True), get_lpb_features(feature_patch[:,:,channel]))))
        #hog_features.append(np.zeros(432))   # <--- short out
    return np.ravel(hog_features)

def scalePatch(patch):
    global X_scaler # use previously defined scaler
    X= patch.astype(np.float64).reshape((1, -1))                       
    # Apply the scaler to X
    return np.array(X_scaler.transform(X))

#_______________________________________________________________________________
# Circular queuing class made with Numpy

class NQ():
    def __init__(self,qlen,shape,dtype=np.uint8):
        self.data= np.empty(shape=shape,dtype=dtype)
        self.qlen= qlen
    def put(self,a):
        self.data= np.append(self.data,a, axis=0)
        if len(self.data)>self.qlen: self.drop()
    def peek(self):
        return self.data[len(self.data)-1]
    def drop(self):
        self.data= np.delete(self.data, 0, axis=0)
    def getAvg(self):
        newList= np.copy(self.data[0])
        newList[:]= np.sum(self.data[:], axis=0)/float(len(self.data))
        return newList

def renderRig(img, cx,cy, width,height, clr, strokeWeight):
    pts=np.array([[0.75,0.1275], [0.4866666667,0.085], [0.286666667,0.1025], [0,0.3875], [0,0.645], [0.47,0.9275], [1,0.8825], [1,0.38]])
    pts[:,0]= pts[:,0]*width  + cx-width //2
    pts[:,1]= pts[:,1]*height + cy-height//2
    cv2.polylines(img, [pts.astype(np.int32)], True, clr, strokeWeight)

x= np.linspace(0, 14, 15)
y=np.array([766,780,796,814,834,857,882,910,941,976,1014,1057,1103,1155,1214])
coefs1= np.polyfit(x,y,3)
y=np.array([411,410,410,409,409,408,407,406,406,405,403,402,401,398,397])
coefs2= np.polyfit(x,y,2)
y=np.array([14,16,18,20,23,25,28,31,35,38,43,46,52,59,66])
coefs3= np.polyfit(x,y,2)
def transformIdxToScreen(idxFloat):
    global coefs1, coefs2, coefs3
    f1= np.poly1d(coefs1)
    f2= np.poly1d(coefs2)
    f3= np.poly1d(coefs3)
    return f1(idxFloat), f2(idxFloat), f3(idxFloat)

In [7]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap
    
def apply_threshold(heatmap, threshold):
    heatmap[heatmap <= threshold] = 0 # Zero out pixels below the threshold
    return heatmap

def draw_labeled_bboxes(img, labels):
    global frameCount
    # Iterate through all detected cars
#    if labels[1]>2: print(frameCount,labels[1])
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero= (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy= np.array(nonzero[0])
        nonzerox= np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox= ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,150,255), 5)

box_list= [[], []]  # define two box lists, for adjacent lanes
#box_list= NQ(6, (0,2), np.array)

def addBoxToList(idx, x,y,s): #add a screen box for each of the positive cells
    global box_list
    box_list[idx].append( ((x, y), (x+s+5, y+s+5)) ) # make the cells slightly larger so they overlap

def drawData(img):
    global box_list, frameCount
    for i in range(0,2):
        heat= np.zeros_like(img[:,:,0]).astype(np.float)
        # Add heat to each box in box list
        heat= add_heat(heat,box_list[i])

        # Apply threshold to help remove false positives
        heat= apply_threshold(heat,2)

        # Find final boxes from heatmap using label function
        labels= label(heat)  #label(heat)
        #if frameCount==246:
        #    plt.figure(figsize=(16,9))
        #    plt.imshow(np.clip(heat, 0, 255), cmap='hot')
        #    plt.show()
        draw_labeled_bboxes(img, labels)
        
        # Clean up the list
        boxQLEN= 20
        if len(box_list[i])>boxQLEN: # keep the last n boxes
            top= len(box_list[i])-boxQLEN
            for j in range(0,top): # circular queue compatible with code from lecture notes 
                del box_list[i][0]
    

In [9]:
# Batch version
def resizePatch(img):
    if len(img[0]) != 16:
        if len(img[0])>16:
            patch= cv2.resize(img, (16,16), interpolation=3) #CV_INTER_AREA
        else:
            patch= cv2.resize(img, (16,16), interpolation=2) #CV_INTER_CUBIC
    else:
        patch= img
    return patch

def preparePatch(p):
    patch= resizePatch(p)
    fpatch= getFeaturesFromColorPatch(patch)
    spatch= scalePatch(fpatch)
    return spatch

diagScreen= None
imgGlobal= None
patchBatch= [[], [], [], [], []]  # patch, x, y, s, prediction
predictionBatch= []

def addPatch(p,x,y,s):
    global patchBatch
    patchBatch[0].append(p)
    patchBatch[1].append(x)
    patchBatch[2].append(y)
    patchBatch[3].append(s)

def processStripB(x,y,s):
    global frameCount, imgGlobal
    for i in range(4):
        addPatch(imgGlobal[y:y+s, x:x+s, :],x,y,s)
        y= y+s

def processStripC(i):
    global frameCount, imgGlobal
    x,y,s= transformIdxToScreen(i)
    for i in range(4):
        addPatch(imgGlobal[y:y+s, x:x+s, :],x,y,s)
        y= y+s

# Predominant color assessment
def dominantWhite(img): 
    return (img>90).sum() > (img<=90).sum()

def locateCentroid(twoDimArray, prevCenterTuple):
    global frameCount
    mask= np.zeros((4,15),dtype=np.int32)
    #mask= np.zeros_like(twoDimArray)               # <-----------------------------------------------------

    cv2.ellipse(mask, (prevCenterTuple, (7,4),0), 1,-1)
    #mask= np.array(mask, dtype=np.uint8)
    masked= cv2.bitwise_and(mask, twoDimArray)
    m= cv2.moments(np.array(masked),True)
    return m['m10']/m['m00'], m['m01']/m['m00'] #return centroid float: x,y or col,row

prevCentroid= (17.,2.)
centroidQ= NQ(25, (0,2), np.int)
for i in range(25): # bias average to offscreen 
    centroidQ.put([ np.array([1400,529]) ])
lastDiagScreen= None
rx= 0
ry= 0
rs= 0

def processBatch(baseImage): # send the batch to the GPU for predictions
    global frameCount, patchBatch, predictionBatch, diagScreen, prevCentroid, lastDiagScreen
    global rx,ry,rs
    prepBatch= []
    for i in range( len(patchBatch[0]) ):
        prepBatch.append( preparePatch(patchBatch[0][i]) )
    predictionResults= model.predict( np.array(prepBatch).reshape((60,-1)) )
    patchBatch[4]= np.array(predictionResults>0.9,dtype=np.uint8)
    whiteCarFoundMaybe= False
    for i in range( len(patchBatch[4]) ):
        x= patchBatch[1][i]
        y= patchBatch[2][i]
        s= patchBatch[3][i]
        if patchBatch[4][i]:
            # RULE: if only rows 0 and 1 are positive, put both cells into lane 2 box list (index 0)
            if i%4==0 and patchBatch[4][i] and patchBatch[4][i+1] and not patchBatch[4][i+2] and not patchBatch[4][i+3]:
                addBoxToList(0, x,y,s)
                addBoxToList(0, patchBatch[1][i+1],patchBatch[2][i+1],patchBatch[3][i+1])
            # RULE: AND all cells go to lane 1 box list (index 1)
            addBoxToList(1, x,y,s)

            # Diagnostics: Display the positive cells
            if dominantWhite(patchBatch[0][i]):
                diagScreen[y:y+s-1, x:x+s-1, 0]= 63 # red
                diagScreen[y:y+s-1, x:x+s-1, 1]= 31 # green
                if whiteCarFoundMaybe==False:
                    whiteCarFoundMaybe= True
                    whiteCarLoc= i
            else:
                diagScreen[y:y+s-1, x:x+s-1, 1]= 63 # green

        # Diagnostics: Display the negative cells
        else:
            diagScreen[y:y+s-1, x:x+s-1, 0]= 63   #157  Red
        
        # More training data
        #if predictionResults[i]>0.9: # and predictionResults[i]>0.4:
        #    mpimg.imsave('output_images/cells/'+str(frameCount)+'-'+str(x)+'-'+str(y)+'.PNG',resizePatch(patchBatch[0][i]))
    #Search the neighbors of the white car in all directions
#    if whiteCarFoundMaybe:
#        refM= np.array(patchBatch[4].reshape((4,15),order='F'), dtype=np.uint8) # reference data matrix
#        whiteCarLeft= 0 # assume maxes
#        whiteCarRight= 14
#        whiteCarTop= 0
#        whiteCarBottom= 3
#        if whiteCarLoc>4 and whiteCarLoc<56 and predictionBatch[i+4]>0.5 and predictionBatch[i-4]>0.5:

        
    if frameCount>=6000:  #  678 is when Black car arrives in project_video
        for i in range(len(patchBatch[4])):
            if dominantWhite(patchBatch[0][i]): patchBatch[4][i]=0 #mask white car for test purposes
        refM= np.array(patchBatch[4].reshape((4,15),order='F'), dtype=np.int32) # reference data matrix
        #refM.strides= (60,4)

        newCentroid= locateCentroid(refM, prevCentroid)

        newCentroid= (newCentroid[0], 1.5) #lock to middle of track

        refX= np.array(patchBatch[1]).reshape((4,15),order='F')
        refY= np.array(patchBatch[2]).reshape((4,15),order='F')
        refS= np.array(patchBatch[3]).reshape((4,15),order='F')
        x= refX[int(newCentroid[1]),round(newCentroid[0])]
        y= refY[int(newCentroid[1]),int(newCentroid[0])]
        #s= refS[int(newCentroid[1]),int(newCentroid[0])]
        _,_,s= transformIdxToScreen(newCentroid[0])
        x+= int( (newCentroid[0] - int(newCentroid[0]) ) * s) # add the fractional portion      s/2 +
        y+= int(s/2 + (newCentroid[1] - int(newCentroid[1]) ) * s)
        #diagScreen[y:y+10, x:x+10, 2]= 255  # small blue 10x10 square
        centroidQ.put([ np.array([x,y]) ])

        cv2.ellipse(diagScreen, (centroidQ.getAvg(), (s*6,s*4),0), (255,255,0), 2) # ellipse
        #cx,cy= centroidQ.getAvg()
        #stroke=3
        #rigHeight=5
        #rigWidth=7
        #if s<31: 
        #    stroke=2
        #    rigHeight=4.5
        #    rigWidth=6
        #if s<20: 
        #    stroke=1
        #    rigHeight=4
        #    rigWidth=6
        #renderRig(diagScreen, cx,cy, s*rigWidth,s*rigHeight, (0,255,0), stroke)
    #    if x+s*3>1280: maxX= 1280
    #    else: maxX= x+s*3
    #    v1= baseImage[y-s*2:y+s*2, x-s*3:maxX]
    #    mask= cv2.inRange(v1, np.array([60,60,60]), np.array([255,255,255]) )
    #    output= cv2.bitwise_and(v1, v1, mask= mask)
    #    baseImage[y-s*2:y+s*2, x-s*3:maxX]= np.invert(np.dstack((mask,mask,mask)))
    #    rx,ry,rs= x,y,s
        prevCentroid= newCentroid
    drawData(diagScreen)
    lastDiagScreen= np.copy(diagScreen)

def redrawBatch(baseImage): # redraw the last batch
    global patchBatch, predictionBatch, diagScreen, lastDiagScreen
#    global rx,ry,rs
#    x,y,s= rx,ry,rs
#    if x+s*3>1280: maxX= 1280
#    else: maxX= x+s*3
#    v1= baseImage[y-s*2:y+s*2, x-s*3:maxX]
#    mask= cv2.inRange(v1, np.array([60,60,60]), np.array([255,255,255]) )
#    output= cv2.bitwise_and(v1, v1, mask= mask)
#    baseImage[y-s*2:y+s*2, x-s*3:maxX]= np.invert(np.dstack((mask,mask,mask)))
    diagScreen= np.copy(lastDiagScreen)
    patchBatch= [[], [], [], [], []]  # patch, x, y, s, prediction
    predictionBatch= []

def process(img):
    global diagScreen, frameCount, imgGlobal
    diagScreen= np.zeros_like(img).astype(np.uint8)  #*127
    imgGlobal= img
    #for i in range(15):
    #if frameCount>=182 and frameCount<=189:
    #if frameCount==0:                  # <-------------------------------------- FRAME
    
    if frameCount%2==0:
        for i in range(0,15):
            processStripC(i) # add patches to batch
        processBatch(img)
    else:
        redrawBatch(img)

    cv2.putText(img, str(frameCount), (1200,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))
    result= cv2.addWeighted(img, 1, diagScreen, 1, 0) # annotate the original
    #result= diagScreen
    frameCount+= 1
    return result

def procVideo(fileName):
    global frameCount
    frameCount= 0
    clip= VideoFileClip(fileName)
    imgName= fileName.split('/')[1]
    project_video_output= 'output_images/'+imgName
    print('Processing video...')
    project_video_clip= clip.fl_image(process)
    t=time.time()
    project_video_clip.write_videofile(project_video_output, audio=False)
    t2 = time.time()
    print(round(t2-t, 2), 'seconds to complete. ', round(frameCount/(t2-t), 2), 'fps' )

box_list= [[],[]]
procVideo('video/project_video.mp4')  #UltraShort2  Black   project_video    Passing  challenge_video

Processing video...
[MoviePy] >>>> Building video output_images/project_video.mp4
[MoviePy] Writing video output_images/project_video.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:25<00:00,  8.94it/s]  0%|▏                                                                                                                          | 2/1261 [00:00<02:15,  9.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video.mp4 

146.68 seconds to complete.  8.6 fps


In [ ]:
#_______________________________________________________________________________
# Vehicle tracking class

class Vehicle():
    def __init__(self, color, left,top, width,height):
        self.color= color
        self.left= left
        self.top= top
        self.width= width
        self.height= height
        self.dx= 0  # relative speed (computed over frames)
        self.dy= 0
        self.refQ= NQ(25,(0,2))   #4 frames containing an array of refdata
        self.centroidX= left+width/2
        self.centroidY= top+height/2
        
    def updateCentroid(self):  #based on reference queue
        #zip through the queue and update location of centroid
        for i in range(len(self.refQ)):
            pass #  data= self.refQ.peek()
        lastX= self.centroidX
        lastY= self.centroidY
        self.centroidX= left+width/2
        self.centroidY= top+height/2
        #also update dx,dy
        self.dx= self.centroidX-lastX
        self.dy= self.centroidY-lastY
    
    #Resolve real-world patches to vehicle model
    #Patch affinity rules:
    #majority should be of one color (with darker lower on the image)
    #if occluded, patches belong to top vehicle
    def addReferenceData(self, dataList):  #dataList: [ [x,y,s,color], ... ]
        #reject/accept data according to rule set
        #modify dataList here
        #majority should be within a 'normal' range wrt distance (either computed or ascertained over time)
        # with consideration for dx,dy
        #if black car, reject white cells
        
        #add resulting data to queue
        self.refQ.put([ dataList ])
    
    def draw(self):
        global diagScreen
        pass # draw something to the screen
        #first, try just the centroid, or the bounding ellipse
        diagScreen[y:y+s-1, x:x+s-1, 2]= 255  # blue

# Example init for 'Black' video
carWhite= Vehicle('White', 862,415, 88,52)
carBlack= Vehicle('Black', 1233,394, 576,343) #width,height is apx (offscreen)



In [ ]:
# Debug cell

master= np.zeros((60),dtype=np.int32)
refM= master.reshape((4,15),order='F')  # does not work without fixing strides: ,order='F'
refM.strides= (60,4)
mask= np.zeros_like(refM) 
print(mask.shape,mask.ndim,mask.strides)
cv2.ellipse(mask, ((7.5,1.5), (7,4),0), 1,-1)
print(mask)


In [ ]:
img2= mpimg.imread('training/vehicles/JWD_vcells/442.PNG')
img3= cv2.resize(img2, (4,4), interpolation=3)
plt.imshow(img2)
plt.show()



In [ ]:
img= mpimg.imread('training/vehicles/JWD_vcells/442.PNG')
def get_texture_features(img, quantization=8, radius=3):
    lbp= np.ravel(local_binary_pattern(img, quantization, radius, method='uniform'))
    return np.mean(lbp.reshape(-1, 4), axis=1)/(quantization+1)
f = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
l= get_texture_features(f,4,3)
print(l.shape)
plt.plot(l)
plt.title('Texture Features')
plt.show()
print(l)


In [13]:
a = np.array([[0,0,1,2,0,0],
              [0,0,0,1,0,0],
              [2,1,0,1,1,0],
              [0,0,0,2,1,0]])
labeled_array, num_features = label(a)
print(num_features)
print(labeled_array)


2
[[0 0 1 1 0 0]
 [0 0 0 1 0 0]
 [2 2 0 1 1 0]
 [0 0 0 1 1 0]]


In [6]:
a = np.zeros((6,6), dtype=np.int)
a[2:4, 2:4] = 1
a[4, 4] = 1
a[:2, :3] = 2
a[0, 5] = 3
print(a)

[[2 2 2 0 0 3]
 [2 2 2 0 0 0]
 [0 0 1 1 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]]


In [27]:
from scipy import ndimage
loc0= ndimage.find_objects(a)[0]
print(a[loc0])
loc1= ndimage.find_objects(a)[1]
print(a[loc1])
listObj= ndimage.find_objects(a)
print(listObj)

[[1 1 0]
 [1 1 0]
 [0 0 1]]
[[2 2 2]
 [2 2 2]]
[(slice(2, 5, None), slice(2, 5, None)), (slice(0, 2, None), slice(0, 3, None)), (slice(0, 1, None), slice(5, 6, None))]


In [26]:
# Accessing the properties of a slice
print(loc0[0].start,loc0[1].start,   loc0[0].stop, loc0[1].stop)
print(loc1)

2 2 5 5
(slice(0, 2, None), slice(0, 3, None))


In [14]:
print(len(box_list[0]))

50
